<a href="https://colab.research.google.com/github/EphraimElgrabli/Machon-Lev-Test-Blender/blob/main/TestMixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - **Machon Lev Test Blender** -

--------------------------------------------
* Idea by [Shira Hoffman](https://www.linkedin.com/in/shira-hoffman/).
* colab version by ZiK0.
___________________________________________

###run this only one time!

In [ ]:
!gdown 1oyvo-tMrInnluYRds9nVnJ-T4cvM1QUh
!gdown 1NJL48SUmSydnL6V_bSFlM818k_-UI0g_

# Import some necessary libraries
!pip install pypdf
!pip install fpdf
!pip install python-bidi

###Click on the play button :>


In [ ]:
from ipywidgets import widgets
from IPython.display import display, HTML
from google.colab import drive, files
from pypdf import PdfReader, PageObject
from fpdf import FPDF
import re
import random
from bidi.algorithm import get_display

#FUNCTIONS
#---------------------------------------------
def rtl_multi_cell(pdf, txt):
    shura = ""
    ret = []
    words = txt.split(" ")
    maxWidth = pdf.w-pdf.r_margin-pdf.x
    #maxWidth = 20
    for word in words:
        shura += word.strip() + " " #cuz "line" have used already
        lineWidth = pdf.get_string_width(shura)
        if lineWidth > maxWidth - 1:
            shura = ' '.join(shura.split(' ')[:-2])
            ret.append(shura)
            shura = word + " "
    ret.append(shura)
    return ret

#MAIN
#---------------------------------------------
# User input section
source = files.upload()
#maxLineLength = 80

# Create a PDF instance and set up its properties
pdf = FPDF()
pdf.add_page()
pdf.add_font('Heebo', '', 'Heebo.ttf', uni=True)
pdf.set_font("Heebo", size=14)

# Add some blank lines to the PDF and a Watermark
pdf.image('Watermark.png', x=45, y=10, w=120, h=0, type='', link='')
for x in range(5):
    pdf.multi_cell(0, 7, "", align='R')

# Read the input PDF
exam = ""
filename = next(iter(source))
newFilename = filename.replace('.pdf', ' (מעורבל).pdf')
pdfPath = filename
pdfRead = PdfReader(pdfPath)

# Extract text from the input PDF
for page in pdfRead.pages:
    exam += page.extract_text()
print(exam)
# Process the extracted text to extract questions and answers
startPosition = exam.find("שאלה מספר", 0)
exam = exam[startPosition::]
exam_lines = exam.split('\n')
#print(exam)
question = None
questions_list = []

for line in exam_lines:
    if line.startswith(("עמוד", "שאלה")):
        if question is not None:
            random.shuffle(question['answers'])
            questions_list.append(question)
        question = {'title': '', 'question': "", 'answers': []}
        if 'שאלה' in line:
            questionTitle = line[line.index('שאלה'):line.index(':') + 1]
            questionTitle += '\n'
            question['title'] += questionTitle
    elif re.match(r'[א-ד]\s*\.', line):
        if question is not None:
            question['answers'].append(line[3:])
    elif question is not None:
        question['question'] += line # Append the line to the question text

# Add the last question after the loop
if question is not None:
    random.shuffle(question['answers'])
    questions_list.append(question)

# Generate the PDF with questions and answers
for question in questions_list:
    #print(question['question'])
    i = 1
    pdf.set_font("Heebo", 'U', size=18)
    pdf.multi_cell(0, 7, get_display(question['title']), align='R')
    pdf.multi_cell(0, 7, "", align='R')

    pdf.set_font("Heebo", size=16)
    toPrint = rtl_multi_cell(pdf, question['question'])
    for subQuestion in toPrint:
        pdf.multi_cell(0, 7, get_display(re.sub("\s\s+" , " ", subQuestion)), align='R')
    pdf.multi_cell(0, 7, "", align='R')

    pdf.set_font("Heebo", size=14)
    for answer in question['answers']:
        pdf.multi_cell(0, 7, get_display(" ." + str(i)), align='R')
        toPrint = rtl_multi_cell(pdf, answer)
        for subQuestion in toPrint:
            pdf.multi_cell(0, 7, get_display(re.sub("\s\s+" , " ", subQuestion)), align='R')
        i += 1

    pdf.set_font("Heebo", size=12)
    pdf.multi_cell(0, 7, "", align='R')
    pdf.multi_cell(0, 7, "( ^v^)b - Just one more to get 100!", align='C')
    pdf.multi_cell(0, 7, "", align='R')

# Save the generated PDF
pdf.output(newFilename)

# Create a download button for the generated PDF
def download_file(_):
    files.download(newFilename)

download_button = widgets.Button(description=' -Download- ')
download_button.style.button_color = 'green'
download_button.layout.width = '400px'
download_button.layout.height = '50px'
download_button.on_click(download_file)
display(download_button)